# Часть 1 Бустинг (5 баллов)

В этой части будем предсказывать зарплату data scientist-ов в зависимости  от ряда факторов с помощью градиентного бустинга.

В датасете есть следующие признаки:



* work_year: The number of years of work experience in the field of data science.

* experience_level: The level of experience, such as Junior, Senior, or Lead.

* employment_type: The type of employment, such as Full-time or Contract.

* job_title: The specific job title or role, such as Data Analyst or Data Scientist.

* salary: The salary amount for the given job.

* salary_currency: The currency in which the salary is denoted.

* salary_in_usd: The equivalent salary amount converted to US dollars (USD) for comparison purposes.

* employee_residence: The country or region where the employee resides.

* remote_ratio: The percentage of remote work offered in the job.

* company_location: The location of the company or organization.

* company_size: The company's size is categorized as Small, Medium, or Large.

In [2]:
import pandas as pd
from sklearn.compose import ColumnTransformer

df = pd.read_csv("ds_salaries.csv")
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


## Задание 1 (0.5 балла) Подготовка



*   Разделите выборку на train, val, test (80%, 10%, 10%)
*   Выдерите salary_in_usd в качестве таргета
*   Найдите и удалите признак, из-за которого возможен лик в данных


In [3]:
from sklearn.model_selection import train_test_split

X = df.drop(['salary_in_usd', 'salary', 'salary_currency'], axis=1)
y = df['salary_in_usd']
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.20, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42)

In [4]:
# df.head()

## Задание 2 (0.5 балла) Линейная модель


*   Закодируйте категориальные  признаки с помощью OneHotEncoder
*   Обучите модель линейной регрессии
*   Оцените  качество через MAPE и RMSE


In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd
import time

categorical_features = ['experience_level', 'employment_type',
                        'job_title', 'employee_residence',
                        'company_location', 'company_size']

numerical_features = ['work_year', 'remote_ratio']

onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output = False)
X_train_cat = onehot_encoder.fit_transform(X_train[categorical_features])
X_val_cat = onehot_encoder.transform(X_val[categorical_features])
# display(df.head())

X_train_num = X_train[numerical_features].to_numpy()
X_val_num = X_val[numerical_features].to_numpy()
X_train_itog = np.column_stack([X_train_cat, X_train_num])
X_val_itog = np.column_stack([X_val_cat, X_val_num])

model = LinearRegression()

start_train = time.time()
model.fit(X_train_itog, y_train)
train_time = time.time() - start_train
print(f'время фита: {train_time} мс')

start_train = time.time()
y_pred = model.predict(X_val_itog)
train_time = time.time() - start_train
print(f'время предикта: {train_time} мс')

print('MAPE: ', mean_absolute_percentage_error(y_val, y_pred))
print('RMSE: ', np.sqrt(mean_squared_error(y_val, y_pred)))

время фита: 0.07458281517028809 мс
время предикта: 0.0010700225830078125 мс
MAPE:  0.43702845780712796
RMSE:  48028.6442270576


## Задание 3 (0.5 балла) XGboost

Начнем с библиотеки xgboost.

Обучите модель `XGBRegressor` на тех же данных, что линейную модель, подобрав оптимальные гиперпараметры (`max_depth, learning_rate, n_estimators, gamma`, etc.) по валидационной выборке. Оцените качество итоговой модели (MAPE, RMSE), скорость обучения и скорость предсказания.

In [6]:
from xgboost.sklearn import XGBRegressor
import time

params = {
    'max_depth' : 6,
    'learning_rate' : 0.01,
    'n_estimators' : 1000,
    'gamma' : 1,
    # 'max_leaves' : 0,
    'n_jobs' : 4,
}

modelXGB = XGBRegressor(**params)

start_train = time.time()
modelXGB.fit(X_train_itog, y_train)
train_time = time.time() - start_train

print(f'время фита: {train_time} мс')

start_train = time.time()
y_pred_XGB = modelXGB.predict(X_val_itog)
train_time = time.time() - start_train

print(f'время предикта: {train_time} мс')



время фита: 1.417572021484375 мс
время предикта: 0.011137723922729492 мс


In [7]:
print('MAPE: ', mean_absolute_percentage_error(y_val, y_pred_XGB))
print('RMSE: ', np.sqrt(mean_squared_error(y_val, y_pred_XGB)))

MAPE:  0.3810010552406311
RMSE:  45902.19724588356


Окей, мы видим, что MAPE и RMSE частично упали, что хорошо для XGBoost. Мы моежм сказать, что здеьс он справился качественнее. Однако, время в сравнении с линрегом выросло почти в 25-30(!!!) раз с 0,2≈ мс до 5≈ мс. Время предикта увеличилось в 15≈ раз с 0.001≈ до 0.01≈ мс. Изменение параметров шло с целью снизить MAPE и RMSE.

## Задание 4 (1 балл) CatBoost

Теперь библиотека CatBoost.

Обучите модель `CatBoostRegressor`, подобрав оптимальные гиперпараметры (`depth, learning_rate, iterations`, etc.) по валидационной выборке. Оцените качество итоговой модели (MAPE, RMSE), скорость обучения и скорость предсказания.

In [10]:
# !pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.6 MB/s eta 0:00:00


In [11]:
from catboost import CatBoostRegressor

params = {
    'depth': 6,
    'learning_rate': 0.03,
    'iterations': 1000,
    'l2_leaf_reg': 3,
    'eval_metric': 'RMSE',
    'verbose' : 150
}

modelCB = CatBoostRegressor(**params)

start_train = time.time()
modelCB.fit(X_train_itog, y_train)
train_time_CBfit = time.time() - start_train


start_train = time.time()
y_pred_CB = modelCB.predict(X_val_itog)
train_time_CBpredict = time.time() - start_train


0:	learn: 62445.5871218	total: 52.2ms	remaining: 52.1s
150:	learn: 47596.6508514	total: 398ms	remaining: 2.24s
300:	learn: 46016.5835133	total: 762ms	remaining: 1.77s
450:	learn: 44825.0387458	total: 1.06s	remaining: 1.3s
600:	learn: 44065.6201158	total: 1.25s	remaining: 831ms
750:	learn: 43508.9372975	total: 1.44s	remaining: 477ms
900:	learn: 43090.1018727	total: 1.62s	remaining: 178ms
999:	learn: 42869.3671149	total: 1.74s	remaining: 0us


In [12]:
print(f'время фита: {train_time_CBfit} мс')
print(f'время предикта: {train_time_CBpredict} мс')
print()
print('MAPE: ', mean_absolute_percentage_error(y_val, y_pred_CB))
print('RMSE: ', np.sqrt(mean_squared_error(y_val, y_pred_CB)))

время фита: 2.4226603507995605 мс
время предикта: 0.03348398208618164 мс

MAPE:  0.38214006212831403
RMSE:  45594.56598399118


Модель справилась побыстрее(именно на фите, предикт все же медленнее), чем XGBoost и показала примерно схожие результаты с ним. Возможно, есть смысл еще повидоизменять параметры, чтобы добиться альтернативных вариантов, но пока CatBoost именно на этих данных

Гиперпараметры были подобраны вручную, основываясь на валид. выборке. (ну и переборе)




Для применения catboost моделей не обязательно сначала кодировать категориальные признаки, модель может кодировать их сама. Обучите catboost с подбором оптимальных гиперпараметров снова, используя pool для передачи данных в модель с указанием какие признаки категориальные, а какие нет с помощью параметра cat_features. Оцените качество и время. Стало ли лучше?

In [13]:
from catboost import Pool

train_pool = Pool(data = X_train, label = y_train, cat_features=categorical_features)
val_pool = Pool(data = X_val, label = y_val, cat_features=categorical_features)


params = {
    'depth': 6,
    'learning_rate': 0.03,
    'iterations': 150,
    'l2_leaf_reg': 3,
    'eval_metric': 'RMSE',
    'verbose' : 150
}

modelCBp = CatBoostRegressor(**params)

start_time = time.time()
modelCBp.fit(train_pool, eval_set=val_pool)
train_time = time.time() - start_time
print(f'время фита: {train_time} мс')


start_time = time.time()
modelCBp.predict(val_pool)
train_time = time.time() - start_time
print(f'время предикта: {train_time} мс')

print('MAPE: ', mean_absolute_percentage_error(y_val, y_pred_CB))
print('RMSE: ', np.sqrt(mean_squared_error(y_val, y_pred_CB)))

0:	learn: 62726.3778767	test: 61310.8036909	best: 61310.8036909 (0)	total: 1.83ms	remaining: 273ms
149:	learn: 47472.3693411	test: 45949.7023818	best: 45948.8450589 (145)	total: 290ms	remaining: 0us

bestTest = 45948.84506
bestIteration = 145

Shrink model to first 146 iterations.
время фита: 0.3377702236175537 мс
время предикта: 0.0008246898651123047 мс
MAPE:  0.38214006212831403
RMSE:  45594.56598399118


**Ответ:** Модель стала гораздо быстрее. MAPE и RMSE оказались на одном уровне.

Комментарии к предыдущим шагам: Модель справилась побыстрее(именно на фите, предикт все же медленнее), чем XGBoost и показала примерно схожие результаты с ним. Возможно, есть смысл еще повидоизменять параметры, чтобы добиться альтернативных вариантов, но пока CatBoost именно на этих данных

Гиперпараметры были подобраны вручную, основываясь на валид. выборке. (ну и переборе)




## Задание 5 (0.5 балла) LightGBM

И наконец библиотека LightGBM - используйте `LGBMRegressor`, снова подберите гиперпараметры, оцените качество и скорость.


In [32]:
from lightgbm import LGBMRegressor


params = {
    'max_depth' : 10,
    'learning_rate' : 0.04,
    'n_estimators' : 500,
    'num_leaves': 50,
    'verbosity': -1 #я не выдержу еще столько логов в жизни
}

modelLGB = LGBMRegressor(**params)


start_time = time.time()
modelLGB.fit(X_train_itog, y_train, eval_set=[(X_val_itog, y_val)])
train_time = time.time() - start_time
print(f'время фита: {train_time} мс')

start_time = time.time()
y_pred_LGB=modelLGB.predict(X_val_itog)
train_time = time.time() - start_time
print(f'время предикта: {train_time} мс')

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


время фита: 0.49227404594421387 мс
время предикта: 0.03130841255187988 мс


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [29]:
print('MAPE: ', mean_absolute_percentage_error(y_val, y_pred_CB))
print('RMSE: ', np.sqrt(mean_squared_error(y_val, y_pred_CB)))

MAPE:  0.38214006212831403
RMSE:  45594.56598399118


## Задание 6 (2 балла) Сравнение и выводы

Сравните модели бустинга и сделайте про них выводы, какая из моделей показала лучший/худший результат по качеству, скорости обучения и скорости предсказания? Как отличаются гиперпараметры для разных моделей?

**Ответ:** # Итак, ну поехали. По качеству лучше всех себя показали яндексовский CatBoost и XGBoost, а вот хуже всех справился линрег. Оно и понятно, линрег не может и не будет учитывать сложные зависимости и нелинейнсти, он просто проводит прямую линию через данные. Проверяли бы ирисы, линрег был бы абсолютно нулевым, в то время как другие модели показали бы более приближенные к реальности результаты.

Касательно скорости обучения, линрег наоборот всех обыграл и почти мгновенно все решил. Ему нет необходимости тратить много ресурсов, просто перемножить матрицы. Самые медленные были XGBoost и CatBoost. Оно и понятно, он строят много деревьев и пытаются подстроитьься плд сложные нелинейности.
LightGBM золотая середина, он средний и по скорости и по качеству.

CatBoost оказался чуть быстрее, когда сам кодировал катег. признаки, вероятно у него оптимизрованы эти алгоритмы (а мне меньше писать кода, прекрасно).

По поводу гиперпараметров, в XGBoost и CatBoost например делается упор на регуляризацию, а вот в LightGBM по количеству листьев. Но в остальном параметры схожи, особенно у в XGBoost и CatBoost

# Часть 2 Кластеризация (5 баллов)

Будем работать с данными о том, каких исполнителей слушают пользователи музыкального сервиса.

Каждая строка таблицы - информация об одном пользователе. Каждый столбец - это исполнитель (The Beatles, Radiohead, etc.)

Для каждой пары (пользователь, исполнитель) в таблице стоит число - доля прослушивания этого исполнителя этим пользователем.


In [51]:
import pandas as pd
ratings = pd.read_excel("https://github.com/evgpat/edu_stepik_rec_sys/blob/main/datasets/sample_matrix.xlsx?raw=true", engine='openpyxl')
ratings.head()

,user,the beatles,radiohead,deathcab for cutie,coldplay,modest mouse,sufjan stevens,dylan. bob,red hot clili peppers,pink fluid,...,municipal waste,townes van zandt,curtis mayfield,jewel,lamb,michal w. smith,群星,agalloch,meshuggah,yellowcard
0,0,NaN,0.020417,NaN,NaN,NaN,NaN,NaN,0.030496,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,0.184962,0.024561,NaN,NaN,0.136341,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,0.028635,NaN,NaN,NaN,0.024559,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0.043529,0.086281,0.034590,0.016712,0.015935,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Будем строить кластеризацию исполнителей: если двух исполнителей слушало много людей примерно одинаковую долю своего времени (то есть векторы близки в пространстве), то, возможно исполнители похожи. Эта информация может быть полезна при построении рекомендательных систем.

## Задание 1 (0.5 балла) Подготовка

Транспонируем матрицу ratings, чтобы по строкам стояли исполнители.


In [65]:
ratings_t = ratings.set_index('user').T
ratings_t

user,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
the beatles,NaN,NaN,NaN,NaN,0.043529,NaN,NaN,NaN,0.093398,0.017621,...,NaN,NaN,0.121169,0.038168,0.007939,0.017884,NaN,0.076923,NaN,NaN
radiohead,0.020417,0.184962,NaN,NaN,0.086281,0.006322,NaN,NaN,NaN,0.019156,...,0.017735,NaN,NaN,NaN,0.011187,NaN,NaN,NaN,NaN,NaN
deathcab for cutie,NaN,0.024561,0.028635,NaN,0.034590,NaN,NaN,NaN,NaN,0.013349,...,0.121344,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.027893
coldplay,NaN,NaN,NaN,NaN,0.016712,NaN,NaN,NaN,NaN,NaN,...,0.217175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
modest mouse,NaN,NaN,NaN,NaN,0.015935,NaN,NaN,NaN,NaN,0.030437,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
michal w. smith,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
群星,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
agalloch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
meshuggah,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Выкиньте строку под названием `user`.

In [66]:
ratings_t = ratings.T.drop('user', axis=0)
ratings_t

,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
the beatles,NaN,NaN,NaN,NaN,0.043529,NaN,NaN,NaN,0.093398,0.017621,...,NaN,NaN,0.121169,0.038168,0.007939,0.017884,NaN,0.076923,NaN,NaN
radiohead,0.020417,0.184962,NaN,NaN,0.086281,0.006322,NaN,NaN,NaN,0.019156,...,0.017735,NaN,NaN,NaN,0.011187,NaN,NaN,NaN,NaN,NaN
deathcab for cutie,NaN,0.024561,0.028635,NaN,0.034590,NaN,NaN,NaN,NaN,0.013349,...,0.121344,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.027893
coldplay,NaN,NaN,NaN,NaN,0.016712,NaN,NaN,NaN,NaN,NaN,...,0.217175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
modest mouse,NaN,NaN,NaN,NaN,0.015935,NaN,NaN,NaN,NaN,0.030437,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
michal w. smith,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
群星,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
agalloch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
meshuggah,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


В таблице много пропусков, так как пользователи слушают не всех-всех исполнителей, чья музыка представлена в сервисе, а некоторое подмножество (обычно около 30 исполнителей)


Доля исполнителя в музыке, прослушанной  пользователем, равна 0, если пользователь никогда не слушал музыку данного музыканта, поэтому заполните пропуски нулями.



In [70]:
ratings_t = ratings[!ratings['user']].fillna(0)
ratings_t.head()

SyntaxError: invalid syntax (ipython-input-70-2034984325.py, line 1)

## Задание 2 (0.5 балла) Первая кластеризация

Примените KMeans с 5ю кластерами, сохраните полученные лейблы

Выведите размеры кластеров. Полезной ли получилась кластеризация? Почему KMeans может выдать такой результат?

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 3 (0.5 балла) Объяснение результатов

При кластеризации получилось $\geq 1$ кластера размера 1. Выведите исполнителей, которые составляют такие кластеры. Среди них должна быть группа The Beatles.

In [ ]:
# -- YOUR CODE HERE --

Изучите данные, почему именно The Beatles выделяется?

Подсказка: посмотрите на долю пользователей, которые слушают каждого исполнителя, среднюю долю прослушивания.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 4 (0.5 балла) Улучшение кластеризации

Попытаемся избавиться от этой проблемы: нормализуйте данные при помощи `normalize`.

In [ ]:
from sklearn.preprocessing import normalize

# -- YOUR CODE HERE --

Примените KMeans с 5ю кластерами на преобразованной матрице, посмотрите на их размеры. Стало ли лучше? Может ли кластеризация быть полезной теперь?

In [ ]:
# -- YOUR CODE HERE --

**Ответ** # -- YOUR ANSWER HERE --

## Задание 5 (1 балл) Центроиды

Выведите для каждого кластера названия топ-10 исполнителей, ближайших к центроиду по косинусной мере. Проинтерпретируйте результат. Что можно сказать о смысле кластеров?

In [ ]:
from scipy.spatial.distance import cosine


centroids = km.cluster_centers_

# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 6 (1 балл) Визуализация

Хотелось бы как-то визуализировать полученную кластеризацию. Постройте точечные графики `plt.scatter` для нескольких пар признаков исполнителей, покрасив точки в цвета кластеров. Почему визуализации получились такими? Хорошо ли они отражают разделение на кластеры? Почему?

In [ ]:
import matplotlib.pyplot as plt

# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Для визуализации данных высокой размерности существует метод t-SNE (стохастическое вложение соседей с t-распределением). Данный метод является нелинейным методом снижения размерности: каждый объект высокой размерности будет моделироваться объектов более низкой (например, 2) размерности таким образом, чтобы похожие объекты моделировались близкими, непохожие - далекими с большой вероятностью.

Примените `TSNE` из библиотеки `sklearn` и визуализируйте полученные объекты, покрасив их в цвета их кластеров

In [ ]:
from sklearn.manifold import TSNE

# -- YOUR CODE HERE --

## Задание 7 (1 балл) Подбор гиперпараметров

Подберите оптимальное количество кластеров (максимум 100 кластеров) с использованием индекса Силуэта. Зафиксируйте `random_state=42`

In [ ]:
from sklearn.metrics import silhouette_score

# -- YOUR CODE HERE --

Выведите исполнителей, ближайших с центроидам (аналогично заданию 5). Как соотносятся результаты? Остался ли смысл кластеров прежним? Расскажите про смысл 1-2 интересных кластеров, если он изменился и кластеров слишком много, чтобы рассказать про все.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Сделайте t-SNE визуализацию полученной кластеризации.

In [ ]:
# -- YOUR CODE HERE --

Если кластеров получилось слишком много и визуально цвета плохо отличаются, покрасьте только какой-нибудь интересный кластер из задания выше (`c = (labels == i)`). Хорошо ли этот кластер отражается в визуализации?

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --